In [ ]:
import requests
import logging
import base64
import pandas as pd
import json

class CoreDataService:
    """
    Clase para conectar y recuperar datos de SAP CDS views.
    
    Attributes:
        base_url (str): URL base para la conexión con la API.
        batch_mode (bool): Indica si las consultas deben ser procesadas en lotes.
        batch_size (int): Tamaño de cada lote si batch_mode es True.
    """
    
    def __init__(self, base_url, batch_mode=True, batch_size=150):
        """
        Inicializa la instancia con la configuración de conexión necesaria.
        
        Args:
            base_url (str): URL base para la conexión con la API.
            batch_mode (bool): Habilita o deshabilita la consulta en lotes.
            batch_size (int): Define el tamaño de cada lote para la consulta.
        """
        self.base_url = base_url
        self.session = requests.Session()
        self.batch_mode = batch_mode
        self.batch_size = batch_size
        logging.info(f"CoreDataService initialized with base URL: {base_url}")

    def query_cds(self, materials, field, from_cds, column_name):
        """
        Consulta datos según la configuración de batch_mode y maneja errores por lote o por material.
        
        Args:
            materials (list): Lista de materiales para consulta.
            field (str): Campo seleccionado para el query.
            from_cds (str): Nombre de la vista CDS desde donde se consulta.
            column_name (str): Nombre de la columna para el filtro.
        
        Returns:
            tuple: Un DataFrame con todos los resultados concatenados y un diccionario de errores.
        """
        results = []
        errors = {}
        if self.batch_mode:
            for i in range(0, len(materials), self.batch_size):
                batch = materials[i:i + self.batch_size]
                result, batch_errors = self.process_batch(batch, field, from_cds, column_name)
                if result is not None:
                    results.append(result)
                if batch_errors:
                    errors.update(batch_errors)
        else:
            for material in materials:
                try:
                    result = self.process_individual(material, field, from_cds, column_name)
                    if result is not None and not result.empty:
                        results.append(result)
                except Exception as e:
                    logging.error(f"Error processing material {material}: {str(e)}")
                    errors[material] = str(e)

        return pd.concat(results, ignore_index=True) if results else pd.DataFrame(), errors

    def process_batch(self, batch, field, from_cds, column_name):
        """
        Procesa un lote de materiales y devuelve los resultados y errores.
        """
        json_query = self.build_query(batch, field, from_cds, column_name)
        try:
            # Asumiendo que transform_json_input_to_df solo devuelve un DataFrame y no una tupla
            result = self.transform_json_input_to_df(json_query)
            return result, {}  # Devuelve un diccionario vacío de errores si todo sale bien
        except Exception as e:
            logging.error(f"Error processing batch: {str(e)}")
            # Devuelve un DataFrame vacío y un diccionario con los errores correspondientes al lote
            return None, {mat: str(e) for mat in batch}

    def process_individual(self, material, field, from_cds, column_name):
        """
        Procesa un único material y devuelve el resultado.
        """
        json_query = self.build_query([material], field, from_cds, column_name)
        return self.transform_json_input_to_df(json_query)

    def transform_json_input_to_df(self, json_data):
        """
        Envía un query JSON a la API y retorna un DataFrame.
        """
        base64_data = base64.b64encode(json.dumps(json_data).encode()).decode()
        url = f'{self.base_url}{base64_data}'
        response = self.session.get(url)
        response.raise_for_status()
        return pd.DataFrame(response.json())

    def build_query(self, materials, field, from_cds, column_name):
        """
        Construye el query JSON para la API.
        """

        

        filter_query = " or ".join([f"{column_name} eq '{material}'" for material in materials])
        # return {"FIELD": field, "FROM": from_cds, "WHERE": filter_query}
        return {"FIELD": '*', "FROM": 'ZMMT010ZMMT0072', "WHERE": "ERDAT BETWEEN '20250228' AND '20250310' AND MATNR eq 'PB0063252'"}
        


In [24]:
service = CoreDataService("http://sapapp01.prebel.com.co:8000/services/zws_gl_0001?SELECT=", batch_mode=True)
data, errors = service.query_cds(["*"], "*", "ZMMT010ZMMT0072", "MAKTX")

In [25]:
data

,MANDT,ERDAT,MATNR,MAKTX,MEINS,MAABC,MTART,MARCA,ZNEGOC,LABST,...,METASKU,METAABC,METAMARCA,EXCEMES,STOCKID,EXCESUN,EXCESVR,STATUS,COBERTURA,WAERS
0,400,2025-02-28,PB0063252,ACOND OGX BIO & COLLAGEN 50% FREE 577ML,ST,O,ZMER,034,RETAIL,1616.0,...,0,1.0,0,6,0,1616.0,256216.8,R,3.Entre 180 y 270 dias,COP


In [24]:
data2 = data.sort_values(by='ERDAT', ascending=True)
data2

,MANDT,ERDAT,MATNR,MAKTX,MEINS,MAABC,MTART,MARCA,ZNEGOC,LABST,...,METASKU,METAABC,METAMARCA,EXCEMES,STOCKID,EXCESUN,EXCESVR,STATUS,COBERTURA,WAERS
0,400,2025-02-28,PB0063252,ACOND OGX BIO & COLLAGEN 50% FREE 577ML,ST,O,ZMER,034,RETAIL,1616.000,...,0.0,1.0,0.0,6,0.0,1616.0,256216.80,R,3.Entre 180 y 270 dias,COP
6359,400,2025-02-28,PB0080169,ST AGUA MICELAR JM X 250 ML,ST,B,HALB,052,FPT,27061.000,...,0.0,1.0,3.0,0,109706.8,0.0,0.00,V,1.Menor de 90 dias,COP
6360,400,2025-02-28,PB0080233,TAP COR JMC B450K L330 AN220 AL160,ST,B,VERP,052,FPT,6856.000,...,0.0,1.0,3.0,0,7304.0,0.0,0.00,V,1.Menor de 90 dias,COP
6361,400,2025-02-28,PB0080245,EKOS HID POL PES CASTANHA 75G COL MLF,ST,C,FERT,022,FPT,5394.000,...,0.0,1.0,3.0,0,8160.4,0.0,0.00,V,1.Menor de 90 dias,COP
6362,400,2025-02-28,PB0080342,BASE FACEFINITY MF 3IN1 84 SOFT TOFFEE,ST,O,ZMER,102,RETAIL,0.000,...,0.0,1.0,4.7,0,0.0,0.0,0.00,V,1.Menor de 90 dias,COP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25684,400,2025-04-10,0005100074,VEGESERYL HGP LS 9874,G,A,ROH,017,RETAIL,25775.000,...,0.0,1.0,5.0,12,1999.9,23775.1,95813.65,R,5.Mayor o igual a 360 dias,COP
25683,400,2025-04-10,0005100034,CARROT OIL,KG,B,ROH,007,RETAIL,34.491,...,0.0,1.0,5.5,2,12.1,22.4,90293.73,V,1.Menor de 90 dias,COP
25682,400,2025-04-10,0000004389,LIQUID GERMALL PLUS,KG,C,ROH,104,FPT,616.919,...,0.0,1.0,0.0,9,0.0,616.9,511039.96,R,3.Entre 180 y 270 dias,COP
25680,400,2025-04-10,Y089210000,NEW MARTINIQUE MAN I AR924793,KG,T,ROH,001,RETAIL,28.723,...,0.0,1.0,3.1,35,0.8,27.9,17716.50,R,5.Mayor o igual a 360 dias,COP
